# Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from os import listdir
from os.path import isfile, join

# Testing

In [12]:
df = pd.read_csv("../Dataset/2_Korea_Bakery_Price.csv")

In [13]:
df

,Name,price
0,angbutter,4800
1,plain bread,3500
2,jam,1500
3,ice coffe,4000
4,croissant,3500
5,ice coffe latter,4500
6,tiramisu croissant,4800
7,cacao deep,4000
8,pain au chocolat,3500
9,almond croissant,4000


# Load Data

In [57]:
listDataPretrainedModel = []

In [58]:
def addDataPretrainedModel(path):
    try:
        df = pd.read_excel(path)
    except Exception:
        df = pd.read_csv(path)
    listDataPretrainedModel.append(df)

In [59]:
onlyfiles = [f for f in listdir("../Dataset") if isfile(join("../Dataset", f))]

In [60]:
onlyfiles

['1 Coffe_Sales.csv',
 '10 Europe_Bike_Store_Sales.csv',
 '11 E-commerce_Business_Transaction.csv',
 '12 Liquor_Sales.csv',
 '2 Coffee_Bean_Sales.csv',
 '3 Balaji_Fast_Food_Sales.csv',
 '4 Pizza_Sales.csv',
 '5 Coffee_Shop_Sales.xlsx',
 '6 French_Bakery_Daily_Sales.csv',
 '7 Mexico_Toy_Sales.csv',
 '8 Online_Retail_Sales.csv',
 '9 Electronic_Sales.csv']

In [61]:
for file in onlyfiles:
    pathFile = join('../Dataset/', file)
    addDataPretrainedModel(path=pathFile)

# Preprocessing

In [72]:
df = listDataPretrainedModel[3]

In [74]:
df = df.drop('name', axis=1)

In [76]:
df.to_csv('12 Liquor_Sales.csv')

In [ ]:
listDataPretrainedModel[3]

,date,convenience_store,store,name,address,city,zipcode,store_location,county_number,county,...,vendor_no,vendor,item,description,pack,liter_size,state_btl_cost,btl_price,bottle_qty,total
0,2015-02-19,NaN,2190,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,1460 2ND AVE\nDES MOINES 50314\n(41.6055703350...,77,Polk,...,421,"Sazerac Co., Inc.",64858,Fireball Cinnamon Whiskey Mini Dispenser,1,3000,29.72,44.58,6,267.48
1,2015-02-09,NaN,2190,"Central City Liquor, Inc.",1460 2ND AVE,DES MOINES,50314,1460 2ND AVE\nDES MOINES 50314\n(41.6055703350...,77,Polk,...,421,"Sazerac Co., Inc.",64858,Fireball Cinnamon Whiskey Mini Dispenser,1,3000,29.72,44.58,5,222.90
2,2014-10-30,NaN,2190,Central City Liquor Inc.,1460 2ND AVE,DES MOINES,50314,1460 2ND AVE\nDES MOINES 50314\n(41.6055703350...,77,Polk,...,421,Sazerac Co. Inc.,64858,Fireball Cinnamon Whiskey Mini Dispenser,1,3000,29.72,44.58,6,267.48
3,2014-02-27,NaN,2190,Central City Liquor Inc.,1460 2ND AVE,DES MOINES,50314,1460 2ND AVE\nDES MOINES 50314\n(41.6055703350...,77,Polk,...,421,Sazerac Co. Inc.,64858,Fireball Cinnamon Whiskey Mini Dispenser,1,3000,29.72,44.58,1,44.58
4,2014-02-25,NaN,2190,Central City Liquor Inc.,1460 2ND AVE,DES MOINES,50314,1460 2ND AVE\nDES MOINES 50314\n(41.6055703350...,77,Polk,...,421,Sazerac Co. Inc.,64858,Fireball Cinnamon Whiskey Mini Dispenser,1,3000,29.72,44.58,1,44.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2014-02-12,NaN,2508,Hy-vee Food Store #1/ceda,1843 JOHNSON AVENUE N.W.,CEDAR RAPIDS,52405,1843 JOHNSON AVENUE N.W.\nCEDAR RAPIDS 52405\n...,57,Linn,...,395,Proximo,58868,Jose Cuervo Authentic Strawberry Margarita,6,1750,8.20,12.30,18,221.40
14996,2014-09-09,NaN,2508,Hy-Vee Food Store #1 / Cedar Rapids,1843 JOHNSON AVENUE N.W.,CEDAR RAPIDS,52405,1843 JOHNSON AVENUE N.W.\nCEDAR RAPIDS 52405\n...,57,Linn,...,395,Proximo,58872,Jose Cuervo Authentic Light Margarita Lime,6,1750,8.20,12.30,6,73.80
14997,2014-02-12,NaN,2508,Hy-vee Food Store #1/ceda,1843 JOHNSON AVENUE N.W.,CEDAR RAPIDS,52405,1843 JOHNSON AVENUE N.W.\nCEDAR RAPIDS 52405\n...,57,Linn,...,395,Proximo,58873,Jose Cuervo Authentic Watermelon Margarita,6,1750,8.20,12.30,12,147.60
14998,2014-01-08,NaN,2508,Hy-vee Food Store #1/ceda,1843 JOHNSON AVENUE N.W.,CEDAR RAPIDS,52405,1843 JOHNSON AVENUE N.W.\nCEDAR RAPIDS 52405\n...,57,Linn,...,395,Proximo,58875,Jose Cuervo Golden Margarita,6,1750,10.50,15.74,6,94.44
